## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

## First, I'll compute the camera calibration using chessboard images and apply a distortion

In [1]:
import matplotlib
matplotlib.use('Qt4Agg')
#import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
import numpy as np
import cv2
import glob
import os
%matplotlib inline


In [2]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')
#print (images)

gray = []
img = []

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(5)

# Now I need to calculate the calibration of the camera, given the chessboard object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

undist = []
#Just to visualize I'm going to show undistorted version of each
for fname in images:
    img2 = cv2.imread(fname)
    undist = cv2.undistort(img2, mtx, dist, None, mtx)

    cv2.imshow('Undistorted',undist)
    cv2.waitKey(5)
    
cv2.destroyAllWindows()

# Write to a file for the writeup
cv2.imwrite('camera_cal/annotated.jpg', img)
cv2.imwrite('camera_cal/undistorted.jpg', undist)


True

### Use color transforms, gradients, etc., to create a thresholded binary image.

In [88]:
x_thresh_min = 40
x_thresh_max = 255
y_thresh_min = 80
y_thresh_max = 100
s_thresh_min = 150
s_thresh_max = 255
r_thresh_min = 0
r_thresh_max = 100
h_thresh = (15, 100)
l_thresh = (100, 255)


In [91]:
def undistort(img):
    return cv2.undistort(img, mtx, dist, None, mtx)

def nlxarch(img):
    # Take the saturation channel from HLS color encoding
    r_channel = img[:,:,0]
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    
    # Lets grayscale the image here. May come back and try other grayscales
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    #equ = cv2.equalizeHist(gray)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))
    equ = clahe.apply(gray)
    
    #Sobel X
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    #Sobel Y
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    abs_sobely = np.absolute(sobely) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobely = np.uint8(255*abs_sobely/np.max(abs_sobely))    
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= x_thresh_min) & (scaled_sobel <= x_thresh_max)] = 1   
    
    # Threshold y gradient
    sybinary = np.zeros_like(scaled_sobely)
    sybinary[(scaled_sobely >= y_thresh_min) & (scaled_sobely <= y_thresh_max)] = 1   
    
    # Threshold color channel
    r_binary = np.zeros_like(s_channel)
    r_binary[(r_channel >= r_thresh_min) & (r_channel <= r_thresh_max)] = 1
    
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1
    
    h_binary = np.zeros_like(h_channel)
    h_binary[(h_channel > h_thresh[0]) & (h_channel <= h_thresh[1])] = 1
    
    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel > l_thresh[0]) & (l_channel <= l_thresh[1])] = 1
    
    combined_hsbinary = np.zeros_like(s_binary)
    #combined_hsbinary[(s_binary == 1) & (h_binary == 1) & (l_binary == 1)] = 1# &  (r_binary == 1)] = 1
    combined_hsbinary[(s_binary == 1) & (h_binary == 1)] = 1# &  (r_binary == 1)] = 1

    
    combined_sxybinary = np.zeros_like(sxbinary)
    combined_sxybinary[(sxbinary == 1) & (sybinary == 1)] = 1
    
    xy_color = np.dstack(( np.zeros_like(sxbinary), combined_sxybinary, sxbinary, sybinary))
    
    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

#     # Combine the two binary thresholds
    combined_binary = np.zeros_like(s_channel)
#    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    combined_binary[(combined_hsbinary == 1) | (sxbinary == 1)] = 1
    #combined_binary[(combined_hsbinary == 1) | ( combined_sxybinary == 1)] = 1
    #combined_binary[(combined_hsbinary == 1)] = 1
    
    #combined_binary[(s_binary == 1) | (combined_sxybinary == 1)] = 1
    
    #color_binary = np.dstack(( combined_binary, sxbinary, s_binary)) * 255
#     fig, axeslist = plt.subplots(3,3)
#     axeslist.ravel()[0].imshow(r_binary, cmap=plt.gray())
#     axeslist.ravel()[1].imshow(h_binary, cmap=plt.gray())
#     axeslist.ravel()[2].imshow(l_binary, cmap=plt.gray())
#     axeslist.ravel()[3].imshow(s_binary, cmap=plt.gray())
#     axeslist.ravel()[4].imshow(sxbinary, cmap=plt.gray())
#     axeslist.ravel()[5].imshow(sybinary, cmap=plt.gray())
#     axeslist.ravel()[6].imshow(combined_sxybinary, cmap=plt.gray())
#     plt.tight_layout()
    #plt.show()

    return(combined_binary)
    #return(xy_color)

In [92]:
# We will use a new set of test images, the ones found in the test_images directory

# Make a list of calibration images
images = glob.glob('test_images/*.jpg')
#print(images)
#images = ['test_images\\test4.jpg', 'test_images\\test5.jpg', 'test_images\\test6.jpg']

# Step through the list and search for chessboard corners
for fname in images[-5:]:
    img = cv2.imread(fname)
    undist = undistort(img)
    combined = nlxarch(undist) * 255
    
    # TEMP
    cv2.imshow('undistorted', combined)
    cv2.waitKey(1000)
    
cv2.destroyAllWindows()

### Apply a perspective transform to rectify binary image ("birds-eye view").

In [93]:
src_points = np.float32(
[[205, 720],
 [1110, 720],
 [600, 448],
 [680, 448]]
)

dest_points = np.float32(
[[320, 720],
 [960, 720],
 [320, 0],
 [960, 0]]
)

M = cv2.getPerspectiveTransform(src_points, dest_points)
Minv = cv2.getPerspectiveTransform(dest_points, src_points)


In [94]:
def warp(img):
    warped = cv2.warpPerspective(img, M, (img.shape[1],img.shape[0]), flags=cv2.INTER_LINEAR)
    return(warped)

#def unwarp(img):

In [95]:
images = glob.glob('test_images/*.jpg')
#print(images)

for fname in images:
    img = cv2.imread(fname)
    undist = undistort(img)
    combined = nlxarch(undist)
    warped_img = warp(combined) *255
    
    # TEMP
    cv2.imshow('Warped', warped_img)
    cv2.waitKey(1000)
#cv2.destroyAllWindows()

### Detect lane pixels and fit to find the lane boundary.

In [9]:
# Hold for tuning global variables
# Choose the number of sliding windows
nwindows = 9
# Set the width of the windows +/- margin
margin = 150
# Set minimum number of pixels found to recenter window
minpix = 50

In [10]:
def sliding_window_search(warped_img):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(warped_img[int(warped_img.shape[0]/2):,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((warped_img, warped_img, warped_img))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Set height of windows
    window_height = np.int(warped_img.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = warped_img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = warped_img.shape[0] - (window+1)*window_height
        win_y_high = warped_img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)    

    # Generate x and y values for plotting
    ploty = np.linspace(0, warped_img.shape[0]-1, warped_img.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    #Add the yellow line for the estimated lane geometry
    for i in ploty:
        j = int(i)

        xl_ax = int(left_fitx[j])
        y_ax = int(ploty[j])
        xr_ax = int(right_fitx[j])
        
        if ((xl_ax > 0) and (xl_ax < out_img.shape[1])) and ( (y_ax > 0) and (y_ax < out_img.shape[0]) ):
            out_img[y_ax][xl_ax] = [0, 255, 255]
        if ((xr_ax > 0) and (xr_ax < out_img.shape[1])) and ( (y_ax > 0) and (y_ax < out_img.shape[0]) ):
            out_img[y_ax][xr_ax] = [0, 255, 255] 
    
    return(out_img, ploty, left_fitx, right_fitx)
    

In [96]:
images = glob.glob('test_images/*.jpg')
#print(images)

for fname in images:
    img = cv2.imread(fname)
    undist = undistort(img)
    combined = nlxarch(undist)
    warped_img = warp(combined)

   
    lane_lines, ploty, left_fitx, right_fitx = sliding_window_search(warped_img)
    
    # TEMP
    cv2.imshow('Annotated', lane_lines)
    cv2.waitKey(1000)
    cv2.imwrite('annotated_flattened_test_images/'+fname[12:], lane_lines)
    
cv2.destroyAllWindows()

### Determine the curvature of the lane and vehicle position with respect to center.

In [12]:
# Define conversions in x and y from pixels space to meters
ym_per_pix = 3.0/60#30/720 # meters per pixel in y dimension # I calculated about 60 pixels for one dashed line of 3m, roughly the same
xm_per_pix = 3.7/633 # meters per pixel in x dimension, I measured 633 pixels across the lane

def find_poly(img, ploty, left_fitx, right_fitx): 
    y_eval = np.max(ploty)
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    #print(left_curverad, 'm', right_curverad, 'm')
    return(left_curverad, right_curverad)
# Example values: 632.1 m    626.2 m

def find_offset(img, left_fitx, right_fitx):
    center_of_lane = (right_fitx[-1] - left_fitx[-1])/2 + left_fitx[-1]
    center_of_img = img.shape[1] / 2
    #print(center_of_lane)
    #print(center_of_img)
    offset = (center_of_img - center_of_lane) * xm_per_pix
    #print(offset)
    return (offset)
    

In [13]:
images = glob.glob('test_images/*.jpg')
#print(images)

i = 0

for fname in images:
    img = cv2.imread(fname)
    undist = undistort(img)
    combined = nlxarch(undist)
    warped_img = warp(combined)

   
    lane_lines, ploty, left_fitx, right_fitx = sliding_window_search(warped_img)
    
    left_curverad, right_curverad = find_poly(lane_lines, ploty, left_fitx, right_fitx)
    
    offset = find_offset(lane_lines, left_fitx, right_fitx)
    
    print(left_curverad, right_curverad, offset)
        
    # TEMP
    cv2.imshow('Annotated', lane_lines)
    cv2.waitKey(1000)
    cv2.imwrite('annotated_flattened_test_images/'+fname[12:], lane_lines)
    
    
    
cv2.destroyAllWindows()

334053.940437 7370.1156245 0.00273353996547
4570.2665676 3687.69377153 -0.0094734441456
244.284977662 480.431498864 -0.193316493641
461.451153073 657.957066373 -0.398381938035
731.767377266 613.982488762 -0.110836090186
1886.20665874 506.305938326 -0.276153532442
385.525042373 477.26798272 0.022878518806
1178.44651784 937.097806249 -0.224742396346


### Warp the detected lane boundaries back onto the original image.
### Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [14]:
def draw_on_original(warped, img, ploty, left_fitx, right_fitx):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)   
    return(result)

In [15]:
images = glob.glob('test_images/*.jpg')
#print(images)

i = 0

for fname in images:
    img = cv2.imread(fname)
    undist = undistort(img)
    combined = nlxarch(undist)
    warped_img = warp(combined)

    lane_lines, ploty, left_fitx, right_fitx = sliding_window_search(warped_img)
    
    left_curverad, right_curverad = find_poly(lane_lines, ploty, left_fitx, right_fitx)
    
    offset = find_offset(lane_lines, left_fitx, right_fitx)
    
    img_with_fill = draw_on_original(warped_img, undist, ploty, left_fitx, right_fitx)
    
    radius = (left_curverad + right_curverad)/2
        
        
    text = "Offset: " + "{:3.2f}".format(offset) + " meters"
    text2 = "Radius of Curvature: "+ "{:3.2f}".format(radius/1000) + "(km)" 
    # TEMP
    font = cv2.FONT_HERSHEY_PLAIN
    annotated = cv2.putText(img_with_fill, text, (10,40), font, 2,(255,255,255), 2, cv2.LINE_AA )
    annotated = cv2.putText(annotated, text2, (10,80), font, 2, (255,255,255),2,cv2.LINE_AA)
    cv2.imshow('Annotated', annotated)
    cv2.waitKey(1000)
    cv2.imwrite('annotated_test_images/'+fname[12:], annotated)

cv2.destroyAllWindows()

## Video Architecture

I decided to redo this in another location where I could mess with the preceding code more. Please see video_architecture.ipynb


In [16]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [17]:


# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        #Number of lines to average over
        self.n = 10
        #Number of bad fits in a row
        self.bad_fit_n = 5
    
    def add_fit(self, fit):
        print(fit)
        if fit is not None:
            if self.best_fit is not None:
                self.diffs = abs(self.best_fit - fit)
                if ((self.diffs[0] > 0.01 or \
                    self.diffs[1] > 1.0 or \
                    self.diffs[2] > 100) and \
                    len(self.current_fit) > 0):
                    #This indicates a bad fit and we shouldn't add the fit
                    self.detected = False
                else:
                    self.detected = True
                    self.current_fit.append(fit)
                    if len(self.current_fit) > self.n:
                        self.current_fit = self.current_fit[len(self.current_fit)-self.n]
                    if len(self.current_fit) > 1:
                        self.best_fit = np.average(self.current_fit, axis=0)
                    else:
                        self.best_fit = self.current_fit
            else:
                self.detected = False
                if len(self.current_fit) > 0:
                    self.current_fit = self.current_fit.pop(0)
                if len(self.current_fit) > 0:
                    self.best_fit = np.average(self.current_fit, axis=0)
                    

In [18]:
def search_from_last_frame(warped_img, left_fit, right_fit):
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((warped_img, warped_img, warped_img))*255
    # Assume you now have a new warped binary image 
    # from the next frame of video (also called "binary_warped")
    # It's now much easier to find line pixels!
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
    left_fit[1]*nonzeroy + left_fit[2] + margin))) 

    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
    right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    #Add the yellow line for the estimated lane geometry
    for i in ploty:
        j = int(i)

        xl_ax = int(left_fitx[j])
        y_ax = int(ploty[j])
        xr_ax = int(right_fitx[j])
        
        if ((xl_ax > 0) and (xl_ax < out_img.shape[1])) and ( (y_ax > 0) and (y_ax < out_img.shape[0]) ):
            out_img[y_ax][xl_ax] = [0, 255, 255]
        if ((xr_ax > 0) and (xr_ax < out_img.shape[1])) and ( (y_ax > 0) and (y_ax < out_img.shape[0]) ):
            out_img[y_ax][xr_ax] = [0, 255, 255] 
    
    return(out_img, ploty, left_fitx, right_fitx)

def video_sliding_window_search(warped_img):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(warped_img[int(warped_img.shape[0]/2):,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((warped_img, warped_img, warped_img))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Set height of windows
    window_height = np.int(warped_img.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = warped_img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = warped_img.shape[0] - (window+1)*window_height
        win_y_high = warped_img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)    

    # Generate x and y values for plotting
    ploty = np.linspace(0, warped_img.shape[0]-1, warped_img.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    #Add the yellow line for the estimated lane geometry
    for i in ploty:
        j = int(i)

        xl_ax = int(left_fitx[j])
        y_ax = int(ploty[j])
        xr_ax = int(right_fitx[j])
        
        if ((xl_ax > 0) and (xl_ax < out_img.shape[1])) and ( (y_ax > 0) and (y_ax < out_img.shape[0]) ):
            out_img[y_ax][xl_ax] = [0, 255, 255]
        if ((xr_ax > 0) and (xr_ax < out_img.shape[1])) and ( (y_ax > 0) and (y_ax < out_img.shape[0]) ):
            out_img[y_ax][xr_ax] = [0, 255, 255] 
    
    return(left_fit, right_fit)
    

In [23]:
def process_image(img):
    undist = undistort(img)
    combined = nlxarch(undist)
    warped_img = warp(combined)

#     if not left.detected or not right.detected:
#         lane_lines, ploty, left_fit, right_fit = video_sliding_window_search(warped_img)
#     else:
#         lane_lines, ploty, left_fit, right_fit = search_from_last_frame(warped_img, left.best_fit, right.best_fit)
    
#     left_curverad, right_curverad = find_poly(lane_lines, ploty, left_fitx, right_fitx)
    
#     if (left_curverad - right_curverad) / (min(left_curverad, right_curverad))  >  (min(left_curverad, right_curverad))*0.25:
#         print("lines are very different")
#         left.add_fit(None)
#         right.add_fit(None)
#     else:
#         left.add_fit(left_fitx)
#         right.add_fit(right_fitx)
    lane_lines, ploty, left_fitx, right_fitx = sliding_window_search(warped_img)

    left_curverad, right_curverad = find_poly(lane_lines, ploty, left_fitx, right_fitx)
        
    offset = find_offset(lane_lines, left_fitx, right_fitx)
    
    img_with_fill = draw_on_original(warped_img, undist, ploty, left_fitx, right_fitx)
    
    radius = (left_curverad + right_curverad)/2

#     print(left.best_fit, right.best_fit)

#     left_curverad, right_curverad = find_poly(lane_lines, ploty, left.best_fit, right.best_fit)
#     offset = find_offset(lane_lines, left.best_fit, right.best_fit)
#     img_with_fill = draw_on_original(warped_img, undist, ploty, left.best_fit, right.best_fit)
#     radius = (left_curverad + right_curverad)/2
        
        
    text = "Offset: " + "{:3.2f}".format(offset) + " meters"
    text2 = "Radius of Curvature: "+ "{:3.2f}".format(radius/1000) + "(km)" 
    # TEMP
    font = cv2.FONT_HERSHEY_PLAIN
    annotated = cv2.putText(img_with_fill, text, (10,40), font, 2,(255,255,255), 2, cv2.LINE_AA )
    annotated = cv2.putText(annotated, text2, (10,80), font, 2, (255,255,255),2,cv2.LINE_AA)
    return(annotated)

In [25]:
# left = Line()
# right = Line()


video_output = "output_images/project_video_output_test.mp4"
clip1 = VideoFileClip("project_video.mp4")
clip1_output = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time clip1_output.write_videofile(video_output, audio=False)


[MoviePy] >>>> Building video output_images/project_video_output_test.mp4
[MoviePy] Writing video output_images/project_video_output_test.mp4



  5%|████▎                                                                           | 67/1261 [00:14<04:23,  4.54it/s]


 11%|████████▍                                                                      | 135/1261 [00:29<04:04,  4.60it/s]


 16%|████████████▋                                                                  | 203/1261 [00:44<03:58,  4.44it/s]


 21%|████████████████▉                                                              | 271/1261 [00:59<03:50,  4.30it/s]


...

 86%|███████████████████████████████████████████████████████████████████▏          | 1087/1261 [04:02<00:55,  3.14it/s]


 92%|███████████████████████████████████████████████████████████████████████▍      | 1155/1261 [04:25<00:28,  3.71it/s]


 97%|███████████████████████████████████████████████████████████████████████████▋  | 1223/1261 [04:41<00:07,  4.76it/s]


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [04:48<00:00,  4.74it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_images/project_video_output_test.mp4 

Wall time: 4min 50s


In [21]:
video_output = "output_images/challenge_video_output.mp4"
clip1 = VideoFileClip("challenge_video.mp4")
clip1_output = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time clip1_output.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video output_images/challenge_video_output.mp4
[MoviePy] Writing video output_images/challenge_video_output.mp4


 22%|█████████████████▋                                                              | 107/485 [00:27<01:37,  3.88it/s]

KeyboardInterrupt: 